In [91]:
import sys
sys.path.append('')
import json
import requests
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from pprint import pprint
from config import weather
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import statsmodels.api as sm
from scipy import stats

In [99]:
data0 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag0')
data1 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag1')
data2 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag2')
data3 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag3')
data4 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag4')
data5 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag5')
data0_sample = data0.sample(225)
data1_sample = data1.sample(127)
data2_sample = data2.sample(35)
data3_sample = data3.sample(11)
data4_sample = data4
data5_sample = data5

data = pd.concat([data0_sample, data1_sample, data2_sample, data3_sample, data4_sample, data5_sample])


In [100]:

test = data
test.shape
#test = data.iloc[25:51]

(459, 22)

In [101]:

#putting coordinates into a list
coordinates = test[['Starting Latitude', 'Starting Longitude']]
#print(coordinates)
coord_list = []
for index, row in coordinates.iterrows():
    y = row['Starting Latitude']
    z = row['Starting Longitude']
    a = (y,z)
    coord_list.append(a)


In [102]:
#putting dates into list

start_date = []
end_date = []

for index, row in test[['Year','Month','Day','Date']].iterrows():
    year = row['Year']
    month = row['Month']
    day = row['Day']
    e = datetime(year,month,day)-timedelta(3)
    e= e.strftime('%Y-%m-%d')
    end_date.append(e)
    s = datetime(year,month,day)
    s= s.strftime('%Y-%m-%d')
    start_date.append(s)

In [103]:
# df to add weather data to
f = pd.DataFrame()
f['Magnitude'] = test['Magnitude']
f['Start Date'] = start_date
f['End Date'] = end_date
f['coord'] = coord_list
f['Max Temp (F)'] = ''
f['Min Temp (F)'] = ''
f['Wind Speed (mph)'] = ''
#f['Precip (mm)'] = ''
f['Humidity'] = ''
f['Visibility'] = ''
f['Pressure'] = ''
f['Cloud Cover'] = ''
f['Heat Index (F)'] = ''
f['Dew Point (F)'] = ''
f['Wind Chill (F)'] = ''
f['Wind Gust (mph)'] = ''

In [104]:
for index,row in f.iterrows():
    coord = row['coord']
    start = row['Start Date']
    end = row['End Date']
    tp = '24'

    maxtF = []
    mintF = []
    wind = []
   # precip_list = []
    hum_list = []
    vis_list = []
    pres_list = []
    clou_list = []
    heat_list = []
    dew_list = []
    wind_chill_list = []
    wind_gust_list = []

    url = ('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={0}&q={1},{2}&format=json&date={3}&tp={4}'
          ).format(weather,coord[0],coord[1],start,tp)
    weather_data = requests.get(url).json()
    
    for i in range(0,1):
        max_temp = int(weather_data['data']['weather'][i]['maxtempF'])
        maxtF.append(max_temp)
        min_temp = int(weather_data['data']['weather'][i]['mintempF'])
        mintF.append(min_temp)
        wind_speed = int(weather_data['data']['weather'][i]['hourly'][0]['windspeedMiles'])
        wind.append(wind_speed)
        #precip = weather_data['data']['weather'][i]['hourly'][0]['precipMM']
        #precip_list.append(precip)
        humidity = int(weather_data['data']['weather'][i]['hourly'][0]['humidity'])
        hum_list.append(humidity)
        visibility = int(weather_data['data']['weather'][i]['hourly'][0]['visibility'])
        vis_list.append(visibility)
        pressure = int(weather_data['data']['weather'][i]['hourly'][0]['pressure'])
        pres_list.append(pressure)
        cloud_cover = int(weather_data['data']['weather'][i]['hourly'][0]['cloudcover'])
        clou_list.append(cloud_cover)
        heat_index = int(weather_data['data']['weather'][i]['hourly'][0]['HeatIndexF'])
        heat_list.append(heat_index)
        dew_point = int(weather_data['data']['weather'][i]['hourly'][0]['DewPointF'])
        dew_list.append(dew_point)
        wind_chill = int(weather_data['data']['weather'][i]['hourly'][0]['WindChillF'])
        wind_chill_list.append(wind_chill)
        wind_gust = int(weather_data['data']['weather'][i]['hourly'][0]['WindGustMiles'])
        wind_gust_list.append(wind_gust)
    max_temp_mean = pd.DataFrame({'max':maxtF}).mean()
    min_temp_mean = pd.DataFrame({'min':mintF}).mean()
    wind_speed_mean = pd.DataFrame({'wind_speed':wind}).mean()
    #precip_mean = pd.DataFrame({'precip':precip_list}).mean()
    hum_mean = pd.DataFrame({'hum':hum_list}).mean()
    visib_mean = pd.DataFrame({'vis':vis_list}).mean()
    pres_mean = pd.DataFrame({'pres': pres_list}).mean()
    clou_mean = pd.DataFrame({'cloud':clou_list}).mean()
    heat_index_mean = pd.DataFrame({'heat':heat_list}).mean()
    dew_mean = pd.DataFrame({'dew':dew_list}).mean()
    wind_chill_mean = pd.DataFrame({'wchill':wind_chill_list}).mean()
    wind_gust_mean = pd.DataFrame({'wgust':wind_gust_list}).mean()
    f.loc[index,'Max Temp (F)'] = int(max_temp_mean)
    f.loc[index, 'Min Temp (F)'] = int(min_temp_mean)
    f.loc[index, 'Wind Speed (mph)'] = int(wind_speed_mean)
    #f.loc[index, 'Precip (mm)'] = precip_mean
    f.loc[index, 'Humidity'] = int(hum_mean)
    f.loc[index, 'Visibility'] = int(visib_mean)
    f.loc[index, 'Pressure'] = int(pres_mean)
    f.loc[index, 'Cloud Cover'] = int(clou_mean)
    f.loc[index, 'Heat Index (F)'] = int(heat_index_mean)
    f.loc[index, 'Dew Point (F)'] = int(dew_mean)
    f.loc[index, 'Wind Chill (F)'] = int(wind_chill_mean)
    f.loc[index, 'Wind Gust (mph)'] = int(wind_gust_mean)
    

In [105]:
f.columns

Index(['Magnitude', 'Start Date', 'End Date', 'coord', 'Max Temp (F)',
       'Min Temp (F)', 'Wind Speed (mph)', 'Humidity', 'Visibility',
       'Pressure', 'Cloud Cover', 'Heat Index (F)', 'Dew Point (F)',
       'Wind Chill (F)', 'Wind Gust (mph)'],
      dtype='object')

In [123]:
y = f['Magnitude']
x = f[['Pressure', 'Dew Point (F)',
       'Wind Gust (mph)']]

In [124]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = .2)

In [128]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

In [126]:
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              Magnitude   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     26.93
Date:                Wed, 16 Jan 2019   Prob (F-statistic):           4.73e-16
Time:                        21:06:11   Log-Likelihood:                -768.55
No. Observations:                 459   AIC:                             1545.
Df Residuals:                     455   BIC:                             1562.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              67.7946     13.707     

In [110]:
correlation = f.corr()
correlation

,Magnitude,Max Temp (F),Min Temp (F),Wind Speed (mph),Humidity,Visibility,Pressure,Cloud Cover,Heat Index (F),Dew Point (F),Wind Chill (F),Wind Gust (mph)
Magnitude,1.000000,0.051535,0.075294,0.295792,0.104326,0.038455,-0.308188,0.089619,0.055581,0.137793,0.040435,0.290709
Max Temp (F),0.051535,1.000000,0.737619,-0.024736,-0.510056,0.402046,-0.126241,-0.547964,0.890910,0.470150,0.901627,-0.048174
Min Temp (F),0.075294,0.737619,1.000000,0.033336,-0.040967,0.109187,-0.020883,-0.142100,0.891635,0.796335,0.875146,0.061162
Wind Speed (mph),0.295792,-0.024736,0.033336,1.000000,-0.013512,0.168469,-0.405468,0.163168,-0.068884,-0.045716,-0.086644,0.947225
Humidity,0.104326,-0.510056,-0.040967,-0.013512,1.000000,-0.581860,0.142217,0.739894,-0.283566,0.433243,-0.340190,0.060675
Visibility,0.038455,0.402046,0.109187,0.168469,-0.581860,1.000000,-0.130229,-0.568999,0.239606,-0.148126,0.261856,0.105988
Pressure,-0.308188,-0.126241,-0.020883,-0.405468,0.142217,-0.130229,1.000000,0.068007,-0.036228,0.059704,-0.044945,-0.403209
Cloud Cover,0.089619,-0.547964,-0.142100,0.163168,0.739894,-0.568999,0.068007,1.000000,-0.406391,0.118269,-0.438454,0.212872
Heat Index (F),0.055581,0.890910,0.891635,-0.068884,-0.283566,0.239606,-0.036228,-0.406391,1.000000,0.719590,0.982270,-0.065243
Dew Point (F),0.137793,0.470150,0.796335,-0.045716,0.433243,-0.148126,0.059704,0.118269,0.719590,1.000000,0.676918,0.004710
